## Gathering Data from League of Legends API 

Importing relevant libraries to pull and organize the data

In [ ]:
import requests
import time
import pandas as pd
import json

Setting the API key variable here

In [ ]:
key = 'INSERT API KEY HERE'

We set two variables retrieving static data from the API that was not require an API key. This will be our base data that we use from here on to clean and convert values.

In [ ]:
champion_data = requests.get('https://ddragon.leagueoflegends.com/cdn/6.24.1/data/en_US/champion.json').json()
rank = requests.get('https://na1.api.riotgames.com/lol/league/v4/masterleagues/by-queue/RANKED_SOLO_5x5?api_key=RGAPI-21b08830-5bd8-485b-bd2b-792f84332895').json()

From the summoner rank json, we want to extract the summoner_id for every player retrieved and store them into a list

In [ ]:
master_rank_summoner_id = [] 
for i in rank['entries']:
    master_rank_summoner_id.append(i['summonerId'])

We want to change into another api reference and using the summoner ID we retrieved previously, we want to now retrieve the
encrypted account ID and store those into another list. From here on, the sleep timer will be 1.3 seconds because Riot (company that owns League of Legends) has a limit of the calls for their API. Since we have a development key, we are only allowed to request a max of 100 calls per 120 seconds. That is 1 call per 1.2 seconds but we wanted to give some room for rest so we set it to 1.3 seconds per call.

In [ ]:
master_rank_account_id = []

for i in master_rank_summoner_id:
    url = 'https://na1.api.riotgames.com/lol/summoner/v4/summoners/' + str(i) + '?api_key=' + key
    master_rank_account_id.append(requests.get(url).json()['accountId'])
    time.sleep(1.3)

At this step, we want the match ID which is requires another layer of digging through the API reference. With the last piece
of the puzzle at hand, we use the account IDs to grab the match IDs from every player and store that information into another
list

In [ ]:
master_rank_match_id = []

for i in master_rank_account_id:
    url = 'https://na1.api.riotgames.com/lol/match/v4/matchlists/by-account/' + i + '?api_key=' + key
    if requests.get(url).status_code == 200:
        data = requests.get(url).json()
        for i in data['matches']:
            master_rank_match_id.append(i['gameId'])
    else:
        pass
    time.sleep(1.3)

After all the loops, we do not want to repeat that process again, therefore we saved a text file containing all the match ID
data so we can just call up the IDs without pulling all over again from the top.

In [ ]:
with open("../files/master_rank_match_id.txt", "w") as output:
    output.write(str(master_rank_match_id))

This function opens the text file we generated but because the text file was saved as a single string, we use the eval()
function to convert the information back into a list

In [ ]:
with open('../files/master_rank_match_id.txt', 'r') as f:
    temp = f.read().splitlines()

match_id = eval(temp[0])

As a precaution, we want to remove any duplicate data before we retrieve any player information. this can be simply applied
but setting the list into a set then back into a list to ensure no duplicates remain

In [ ]:
match_id = list(set(match_id))

Finally, we have our match data information and we begin pulling player data per match. PLEASE KEEP IN MIND THIS WILL TAKE A 
VERY LONG TIME. The file containing all player data is stored in the data folder so no need to pull again.

In [ ]:
match_data = []

for i in match_id:
    url = 'https://na1.api.riotgames.com/lol/match/v4/matches/' + str(i) + '?api_key=' + key
    match_data.append(requests.get(url).json())
    time.sleep(1.3)

Since the data retrieve is a huge dictionary, we only want specific data for our analysis. Below we set a placeholder for the dictionary to hold the information we want and append that information to a list created outside the loop. We do this for all players in the file then set it into a dataframe below.

In [ ]:
player_list = []
for match in range(len(match_data)):
    for items in range(5):
        try:
            player_dict_item = {}
            if match_data[match].get('participants'):
                player_dict_item['game_mode'] = match_data[match].get('gameMode')
            player_dict_item['champion'] = match_data[match].get('participants')[items].get('championId')
            player_dict_item['spell_1'] = match_data[match].get('participants')[items].get('spell1Id')
            player_dict_item['spell_2'] = match_data[match].get('participants')[items].get('spell2Id')
            player_dict_item['player_rank'] = match_data[match].get('participants')[items].get('highestAchievedSeasonTier')
            player_dict_item['victory'] = match_data[match].get('participants')[items]['stats'].get('win')
            player_dict_item['kills'] = match_data[match].get('participants')[items]['stats'].get('kills')
            player_dict_item['deaths'] = match_data[match].get('participants')[items]['stats'].get('deaths')
            player_dict_item['assists'] = match_data[match].get('participants')[items]['stats'].get('assists')
            player_dict_item['largest_killing_spree'] = match_data[match].get('participants')[items]['stats'].get('largestKillingSpree')
            player_dict_item['killing_sprees'] = match_data[match].get('participants')[items]['stats'].get('killingSprees')
            player_dict_item['double_kills'] = match_data[match].get('participants')[items]['stats'].get('doubleKills')
            player_dict_item['triple_kills'] = match_data[match].get('participants')[items]['stats'].get('tripleKills')
            player_dict_item['quadra_kills'] = match_data[match].get('participants')[items]['stats'].get('quadraKills')
            player_dict_item['penta_kills'] = match_data[match].get('participants')[items]['stats'].get('pentaKills')
            player_dict_item['total_damage_dealt_to_champions'] = match_data[match].get('participants')[items]['stats'].get('totalDamageDealtToChampions')
            player_dict_item['vision_score'] = match_data[match].get('participants')[items]['stats'].get('visionScore')
            player_dict_item['total_damage_taken'] = match_data[match].get('participants')[items]['stats'].get('totalDamageTaken')
            player_dict_item['gold_earned'] = match_data[match].get('participants')[items]['stats'].get('goldEarned')
            player_dict_item['gold_spent'] = match_data[match].get('participants')[items]['stats'].get('goldSpent')   
            player_dict_item['turret_kills'] = match_data[match].get('participants')[items]['stats'].get('turretKills')
            player_dict_item['inhibitor_kills'] = match_data[match].get('participants')[items]['stats'].get('inhibitorKills')
            player_dict_item['total_minions_killed'] = match_data[match].get('participants')[items]['stats'].get('totalMinionsKilled')
            player_dict_item['wards_placed'] = match_data[match].get('participants')[items]['stats'].get('wardsPlaced')
            player_dict_item['wards_killed'] = match_data[match].get('participants')[items]['stats'].get('wardsKilled')   
            player_dict_item['first_blood_kill'] = match_data[match].get('participants')[items]['stats'].get('firstBloodKill')
            player_dict_item['first_blood_assist'] = match_data[match].get('participants')[items]['stats'].get('firstBloodAssist')
            player_dict_item['first_tower_kill'] = match_data[match].get('participants')[items]['stats'].get('firstTowerKill')
            player_dict_item['first_tower_assist'] = match_data[match].get('participants')[items]['stats'].get('firstTowerAssist')
            player_dict_item['first_inhibitor_kill'] = match_data[match].get('participants')[items]['stats'].get('firstInhibitorKill')
            player_dict_item['first_inhibitor_assist'] = match_data[match].get('participants')[items]['stats'].get('firstInhibitorAssist')  
            player_dict_item['lane'] = match_data[match].get('participants')[items]['timeline'].get('lane')
            player_dict_item['role'] = match_data[match].get('participants')[items]['timeline'].get('role')
            player_dict_item['team_tower_kills'] = match_data[match]['teams'][0]['towerKills']
            player_dict_item['team_baron_kill'] = match_data[match]['teams'][0]['baronKills']
            player_dict_item['team_dragon_kills'] = match_data[match]['teams'][0]['dragonKills']
            player_dict_item['team_rift_herald_kills'] = match_data[match]['teams'][0]['riftHeraldKills']
            

            player_list.append(player_dict_item)
        except:
            pass
        
    for items in range(5,10):
        try:
            player_dict_item = {}
            player_dict_item['game_mode'] = match_data[match].get('gameMode')
            if match_data[match].get('participants')[items]:
                player_dict_item['champion'] = match_data[match].get('participants')[items].get('championId')
            player_dict_item['spell_1'] = match_data[match].get('participants')[items].get('spell1Id')
            player_dict_item['spell_2'] = match_data[match].get('participants')[items].get('spell2Id')
            player_dict_item['player_rank'] = match_data[match].get('participants')[items].get('highestAchievedSeasonTier')
            player_dict_item['victory'] = match_data[match].get('participants')[items]['stats'].get('win')
            player_dict_item['kills'] = match_data[match].get('participants')[items]['stats'].get('kills')
            player_dict_item['deaths'] = match_data[match].get('participants')[items]['stats'].get('deaths')
            player_dict_item['assists'] = match_data[match].get('participants')[items]['stats'].get('assists')
            player_dict_item['largest_killing_spree'] = match_data[match].get('participants')[items]['stats'].get('largestKillingSpree')
            player_dict_item['killing_sprees'] = match_data[match].get('participants')[items]['stats'].get('killingSprees')
            player_dict_item['double_kills'] = match_data[match].get('participants')[items]['stats'].get('doubleKills')
            player_dict_item['triple_kills'] = match_data[match].get('participants')[items]['stats'].get('tripleKills')
            player_dict_item['quadra_kills'] = match_data[match].get('participants')[items]['stats'].get('quadraKills')
            player_dict_item['penta_kills'] = match_data[match].get('participants')[items]['stats'].get('pentaKills')
            player_dict_item['total_damage_dealt_to_champions'] = match_data[match].get('participants')[items]['stats'].get('totalDamageDealtToChampions')
            player_dict_item['vision_score'] = match_data[match].get('participants')[items]['stats'].get('visionScore')
            player_dict_item['total_damage_taken'] = match_data[match].get('participants')[items]['stats'].get('totalDamageTaken')
            player_dict_item['gold_earned'] = match_data[match].get('participants')[items]['stats'].get('goldEarned')
            player_dict_item['gold_spent'] = match_data[match].get('participants')[items]['stats'].get('goldSpent')   
            player_dict_item['turret_kills'] = match_data[match].get('participants')[items]['stats'].get('turretKills')
            player_dict_item['inhibitor_kills'] = match_data[match].get('participants')[items]['stats'].get('inhibitorKills')
            player_dict_item['total_minions_killed'] = match_data[match].get('participants')[items]['stats'].get('totalMinionsKilled')
            player_dict_item['wards_placed'] = match_data[match].get('participants')[items]['stats'].get('wardsPlaced')
            player_dict_item['wards_killed'] = match_data[match].get('participants')[items]['stats'].get('wardsKilled')   
            player_dict_item['first_blood_kill'] = match_data[match].get('participants')[items]['stats'].get('firstBloodKill')
            player_dict_item['first_blood_assist'] = match_data[match].get('participants')[items]['stats'].get('firstBloodAssist')
            player_dict_item['first_tower_kill'] = match_data[match].get('participants')[items]['stats'].get('firstTowerKill')
            player_dict_item['first_tower_assist'] = match_data[match].get('participants')[items]['stats'].get('firstTowerAssist')
            player_dict_item['first_inhibitor_kill'] = match_data[match].get('participants')[items]['stats'].get('firstInhibitorKill')
            player_dict_item['first_inhibitor_assist'] = match_data[match].get('participants')[items]['stats'].get('firstInhibitorAssist')  
            player_dict_item['lane'] = match_data[match].get('participants')[items]['timeline'].get('lane')
            player_dict_item['role'] = match_data[match].get('participants')[items]['timeline'].get('role')
            player_dict_item['team_tower_kills'] = match_data[match]['teams'][1]['towerKills']
            player_dict_item['team_baron_kill'] = match_data[match]['teams'][1]['baronKills']
            player_dict_item['team_dragon_kills'] = match_data[match]['teams'][1]['dragonKills']
            player_dict_item['team_rift_herald_kills'] = match_data[match]['teams'][1]['riftHeraldKills']
           
            
            player_list.append(player_dict_item)
        except:
            pass


This step simply stores the giant list we created into a dataframe

In [ ]:
df = pd.DataFrame(player_list)

As a precaution, we stored the dataframe as a `.csv` file in case we have to restart the notebook and do not want to pull the data again.

In [ ]:
# df.to_csv('../data/all_playhers_combined.csv')

The data has 3 game modes included: Classic, ARAM (All Mid, All Random, and URF (Ultra Rapid Fire). We are only interested in Classic mode because the other two modes are more or less special game modes that are not part of the standard setting. Basically the other 2 modes are for fun and not a standard Summoner's Rift match.

In [ ]:
# classic =  df[df['game_mode'] == 'CLASSIC']

As above, we want to save another `csv` file just in case we get interrupted creating the data.

In [ ]:
# classic.to_csv('../data/classic_games.csv')

At this step, we want to incorporate some formatting before we bring the data into main notebook. The champions column are numerical IDs instead of their actual name. From the data we pull above, we have all the champion information including their name and associated ID. We first create a list of dictionary where the ID is the key and value is the name. Then we store that information into a list for every champion. Next we do the same for summoner spells. The file was a `json` file downloaded from Riot's [Data Dragon](https://developer.riotgames.com/ddragon.html) page. Similar to what we did with the character list, we created a list of dictionarys with the key being the ID and value being the spell name.

In [ ]:
names = list(champion_data['data'].keys())

champion_list = []
champion_dict = {}

for i in names:
    temp = {}
    temp[champion_data['data'][i]['key']] = i
    champion_list.append(temp)

for i in champion_list:
    champion_dict.update(i)
    
with open("capstone/data/spells.json", "r") as read_file:
    spells = json.load(read_file)

spell_names = {}
for i in spells:
    spell_names.update({i['key']: i['name']})

We converted the following columns into string in preparation of replacing all values with the appropriate names.

In [ ]:
classic['champion'] = classic['champion'].astype(str)
classic['spell_1'] = classic['spell_1'].astype(str)
classic['spell_2'] = classic['spell_2'].astype(str)

Here we map the IDs to their names and replace every value in the data with the appropriate name.

In [ ]:
classic['champion'] = classic['champion'].map(champion_dict)
classic['spell_1'] = classic['spell_1'].map(spell_names)
classic['spell_2'] = classic['spell_2'].map(spell_names)

We finally save the file ready for analysis in our main notebook.

In [ ]:
# classic.to_csv('../data/classic_games_added_champion_and_spells.csv')

Here continue [here](League%20of%20Legends.ipynb) for full analysis of the data.